## Read in raw data

In [7]:
# Read in raw data

import os
import pandas as pd

raw_data_path = "../data/raw"

# Store df's in a dictionary
raw_dfs = {}
raw_dfs['holidays_events'] = pd.read_csv(os.path.join(raw_data_path, "holidays_events.csv"))
raw_dfs['oil'] = pd.read_csv(os.path.join(raw_data_path, "oil.csv"))
raw_dfs['stores'] = pd.read_csv(os.path.join(raw_data_path, "stores.csv"))
raw_dfs['train'] = pd.read_csv(os.path.join(raw_data_path, "train.csv"))
raw_dfs['test'] = pd.read_csv(os.path.join(raw_data_path, "test.csv"))

We have a normalized setup of our data

## Basic inspection of data

In [8]:
# Check out high-level info of our df's
def inspect_df(df_name, df):
    print(f" * {df_name}.dtypes: \n{df.dtypes}")
    print(f" * {df_name}.isnull().sum(): \n{df.isnull().sum()}")
    print(f" * {df_name}.shape: {df.shape}")
    print(f" * {df_name}.nunique() : \n{df.nunique()}")

for i, df_name in enumerate(raw_dfs):
    print(f"\n#### {i+1}/{len(raw_dfs)}: {df_name} ####")
    df = raw_dfs[df_name]
    inspect_df(df_name, df)


#### 1/5: holidays_events ####
 * holidays_events.dtypes: 
date           object
type           object
locale         object
locale_name    object
description    object
transferred      bool
dtype: object
 * holidays_events.isnull().sum(): 
date           0
type           0
locale         0
locale_name    0
description    0
transferred    0
dtype: int64
 * holidays_events.shape: (26, 6)
 * holidays_events.nunique() : 
date           24
type            3
locale          3
locale_name    12
description    25
transferred     2
dtype: int64

#### 2/5: oil ####
 * oil.dtypes: 
date           object
dcoilwtico    float64
dtype: object
 * oil.isnull().sum(): 
date          0
dcoilwtico    0
dtype: int64
 * oil.shape: (17, 2)
 * oil.nunique() : 
date          17
dcoilwtico    16
dtype: int64

#### 3/5: stores ####
 * stores.dtypes: 
store_nbr     int64
city         object
state        object
type         object
cluster       int64
dtype: object
 * stores.isnull().sum(): 
store_nbr    0
city  

What we deduce from above:
* Basically no N/A's, only in oil
* No duplicate rows

In [9]:
# Get a better idea of what our data looks like
for i, df_name in enumerate(raw_dfs):
    print(f"\n#### {i+1}/{len(raw_dfs)}: {df_name} ####")
    df = raw_dfs[df_name]

    print(df.head())


#### 1/5: holidays_events ####
         date      type    locale locale_name  \
0  2017-08-10   Holiday  National     Ecuador   
1  2017-08-11  Transfer  National     Ecuador   
2  2017-08-15   Holiday     Local    Riobamba   
3  2017-08-24   Holiday     Local      Ambato   
4  2017-09-28   Holiday     Local      Ibarra   

                              description  transferred  
0           Primer Grito de Independencia         True  
1  Traslado Primer Grito de Independencia        False  
2                   Fundacion de Riobamba        False  
3                     Fundacion de Ambato        False  
4                     Fundacion de Ibarra         True  

#### 2/5: oil ####
         date  dcoilwtico
0  2017-08-09       49.59
1  2017-08-10       48.54
2  2017-08-11       48.81
3  2017-08-14       47.59
4  2017-08-15       47.57

#### 3/5: stores ####
   store_nbr           city                           state type  cluster
0          1          Quito                       Pichinch

## Diagnosis:
* Aren't any duplicate rows
* Only N/A's in 'oil' (will deal with later)
* *Mainly want to set types*
    * Change type of 'date' to pd.datetime
    * Change (other) object types to 'category'


# For total sales

In [10]:
main = raw_dfs['train']
main['date'] = pd.to_datetime(main['date'], format="%Y-%m-%d")
main = main.sort_values('date')

# Group by date and aggregate (e.g., sum/mean) if duplicates exist
daily_sales = main.groupby('date')['sales'].sum().reset_index()  # or .mean(), depending on your goal
daily_sales.set_index('date', inplace=True)

In [11]:
# Define stats
stats = ['mean', 'std', 'min', 'max']

# Rolling over 7 days (now works correctly since index has no duplicates)
rolled = (
    daily_sales[['sales']]
    .rolling('7D', min_periods=1)
    .agg(stats)
)
rolled

sales                                           
                     mean           std           min            max
date                                                                
2017-08-09  734139.674010           NaN  734139.67401  734139.674010
2017-08-10  692763.292990  58515.039201  651386.91197  734139.674010
2017-08-11  737300.102667  87536.204824  651386.91197  826373.722022
2017-08-12  751132.710770  76640.430743  651386.91197  826373.722022
2017-08-13  774034.104110  83831.293697  651386.91197  865639.677471
2017-08-14  771848.821105  75171.812624  651386.91197  865639.677471
2017-08-15  770536.408939  68709.956757  651386.91197  865639.677471

In [12]:
# Step 1: Aggregate sales by store and date (to handle duplicates)
daily_sales = (
    main.groupby(['store_nbr', 'date'])['sales']
    .sum()  # or .mean(), .max(), etc.
    .reset_index()
)

# Step 2: Set 'date' as index (required for time-based rolling)
daily_sales = daily_sales.set_index('date')

# Step 3: Define stats
stats = ['mean', 'std', 'min', 'max']

# Step 4: Group by store_nbr and apply rolling 7D window
rolled = (
    daily_sales.groupby('store_nbr')['sales']
    .rolling('7D', min_periods=1)
    .agg(stats)
)

rolled

mean          std           min           max
store_nbr date                                                             
1         2017-08-09  12583.652000          NaN  12583.652000  12583.652000
          2017-08-10  11709.115500  1236.781379  10834.579000  12583.652000
          2017-08-11   9607.324340  3743.980813   5403.742021  12583.652000
          2017-08-12   9091.399005  3226.397995   5403.742021  12583.652000
          2017-08-13   8053.675604  3632.021793   3902.782000  12583.652000
...                            ...          ...           ...           ...
54        2017-08-11   8785.138000   869.377317   8083.764000   9757.816000
          2017-08-12   8873.773000   731.643766   8083.764000   9757.816000
          2017-08-13   9948.384000  2485.039426   8083.764000  14246.827996
          2017-08-14  10270.819000  2358.839277   8083.764000  14246.827996
          2017-08-15  10613.110285  2336.003471   8083.764000  14246.827996

[378 rows x 4 columns]